<a href="https://www.kaggle.com/code/yeasirmahmud/uber-traffic?scriptVersionId=242077950" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install contextily


importing libraries


In [11]:
uber_df=pd.read_csv("/kaggle/input/uber-fares-dataset/uber.csv")

In [12]:
print(uber_df.head())


   Unnamed: 0                            key  fare_amount  \
0    24238194    2015-05-07 19:52:06.0000003          7.5   
1    27835199    2009-07-17 20:04:56.0000002          7.7   
2    44984355   2009-08-24 21:45:00.00000061         12.9   
3    25894730    2009-06-26 08:22:21.0000001          5.3   
4    17610152  2014-08-28 17:47:00.000000188         16.0   

           pickup_datetime  pickup_longitude  pickup_latitude  \
0  2015-05-07 19:52:06 UTC        -73.999817        40.738354   
1  2009-07-17 20:04:56 UTC        -73.994355        40.728225   
2  2009-08-24 21:45:00 UTC        -74.005043        40.740770   
3  2009-06-26 08:22:21 UTC        -73.976124        40.790844   
4  2014-08-28 17:47:00 UTC        -73.925023        40.744085   

   dropoff_longitude  dropoff_latitude  passenger_count  
0         -73.999512         40.723217                1  
1         -73.994710         40.750325                1  
2         -73.962565         40.772647                1  
3         

In [13]:
print(uber_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         200000 non-null  int64  
 1   key                200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB
None


The is_holiday feature does not significantly explain the variance in fare_amount — neither statistically nor practically.

In [14]:
print(uber_df.head)
print(uber_df.columns)

<bound method NDFrame.head of         Unnamed: 0                            key  fare_amount  \
0         24238194    2015-05-07 19:52:06.0000003          7.5   
1         27835199    2009-07-17 20:04:56.0000002          7.7   
2         44984355   2009-08-24 21:45:00.00000061         12.9   
3         25894730    2009-06-26 08:22:21.0000001          5.3   
4         17610152  2014-08-28 17:47:00.000000188         16.0   
...            ...                            ...          ...   
199995    42598914   2012-10-28 10:49:00.00000053          3.0   
199996    16382965    2014-03-14 01:09:00.0000008          7.5   
199997    27804658   2009-06-29 00:42:00.00000078         30.9   
199998    20259894    2015-05-20 14:56:25.0000004         14.5   
199999    11951496   2010-05-15 04:08:00.00000076         14.1   

                pickup_datetime  pickup_longitude  pickup_latitude  \
0       2015-05-07 19:52:06 UTC        -73.999817        40.738354   
1       2009-07-17 20:04:56 UTC      

In [15]:
weather_df=pd.read_csv("/kaggle/input/weather/weather.csv")

In [16]:
print(weather_df.columns)
print(weather_df.head)


Index(['time', 'temperature_2m (°C)', 'relative_humidity_2m (%)',
       'dew_point_2m (°C)', 'rain (mm)', 'snowfall (cm)', 'precipitation (mm)',
       'pressure_msl (hPa)', 'weather_code (wmo code)',
       'apparent_temperature (°C)', 'wind_speed_10m (km/h)', 'snow_depth (m)',
       'cloud_cover (%)'],
      dtype='object')
<bound method NDFrame.head of                    time  temperature_2m (°C)  relative_humidity_2m (%)  \
0      2009-06-01T00:00                 20.5                        42   
1      2009-06-01T01:00                 18.2                        40   
2      2009-06-01T02:00                 16.4                        38   
3      2009-06-01T03:00                 15.0                        40   
4      2009-06-01T04:00                 13.8                        43   
...                 ...                  ...                       ...   
54043  2015-07-31T19:00                 30.0                        41   
54044  2015-07-31T20:00                 30.0    

In [17]:
weather_df['time'] = pd.to_datetime(weather_df['time'])  # ensure datetime type
weather_df['time_hour'] = weather_df['time'].dt.floor('H')  # round to hour


/tmp/ipykernel_35/1077841376.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  weather_df['time_hour'] = weather_df['time'].dt.floor('H')  # round to hour


In [18]:
uber_df['pickup_hour'] = uber_df['pickup_5min'].dt.floor('H')  # round to hour


KeyError: 'pickup_5min'

In [ ]:
weather_df['time'] = pd.to_datetime(weather_df['time'])
weather_df['time_hour'] = weather_df['time'].dt.floor('H')
weather_df['time_hour'] = weather_df['time_hour'].dt.tz_localize('UTC')


In [ ]:
merged_df = uber_df.merge(
    weather_df,
    left_on='pickup_hour',
    right_on='time_hour',
    how='left'
)


In [ ]:
merged_df.columns


In [ ]:
merged_df.isnull().sum()


In [ ]:
merged_df['bad_weather'] = (
    (merged_df['rain (mm)'] > 0) | 
    (merged_df['snowfall (cm)'] > 0)
)


In [ ]:

merged_df.groupby('bad_weather')['fare_amount'].mean()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(x='bad_weather', y='fare_amount', data=merged_df)
plt.title('Average Fare in Good vs Bad Weather')
plt.xlabel('Bad Weather (True = Rain/Snow)')
plt.ylabel('Average Fare Amount')
plt.show()


In [ ]:
print(merged_df.groupby('bad_weather')['fare_amount'].mean())


## Uber Fare vs Bad Weather

- Average fare in **good weather**: $11.37  
- Average fare in **bad weather**: $11.27  

The difference is minimal, indicating **bad weather doesn’t significantly affect Uber fares** in this data.


In [ ]:
# Count number of rides by bad_weather status
ride_counts = merged_df['bad_weather'].value_counts()

print(ride_counts)


In [ ]:
sns.countplot(x='bad_weather', data=merged_df)
ride_counts = merged_df.groupby('bad_weather').size()
print(ride_counts)

plt.title('Number of Uber Rides: Good vs Bad Weather')
plt.xlabel('Bad Weather (True = Rain/Snow)')
plt.ylabel('Number of Rides')
plt.show()


**Uber rides drop significantly in bad weather:**

- Good weather rides: 173,977  
- Bad weather rides: 26,023  

Rides during rain or snow are much fewer, indicating that bad weather reduces Uber usage.


In [ ]:
import pandas as pd

# Convert pickup_datetime to datetime and floor to the hour (lowercase 'h')
uber_df['pickup_hour'] = pd.to_datetime(uber_df['pickup_datetime']).dt.floor('h')

# Remove timezone info if exists (make timezone naive)
if uber_df['pickup_hour'].dt.tz is not None:
    uber_df['pickup_hour'] = uber_df['pickup_hour'].dt.tz_convert(None)

# Aggregate Uber data by hour: count rides and average fare
uber_hourly = uber_df.groupby('pickup_hour').agg(
    rides_count=('pickup_hour', 'count'),
    avg_fare=('fare_amount', 'mean')
).reset_index()

# For weather, floor time to hour and remove timezone info (make naive)
weather_df['time_hour'] = pd.to_datetime(weather_df['time']).dt.floor('h')
if weather_df['time_hour'].dt.tz is not None:
    weather_df['time_hour'] = weather_df['time_hour'].dt.tz_convert(None)

# Merge Uber and weather on hour, keep all Uber data (left join)
merged_hourly = pd.merge(
    uber_hourly,
    weather_df,
    left_on='pickup_hour',
    right_on='time_hour',
    how='left'
)

# Drop redundant time_hour column
merged_hourly.drop(columns=['time_hour'], inplace=True)

print(merged_hourly.head())
print(f"Uber pickup_hour range: {uber_hourly['pickup_hour'].min()} to {uber_hourly['pickup_hour'].max()}")
print(f"Weather time_hour range: {weather_df['time_hour'].min()} to {weather_df['time_hour'].max()}")
print(f"Merged data range: {merged_hourly['pickup_hour'].min()} to {merged_hourly['pickup_hour'].max()}")


In [ ]:
start_date = weather_df['time_hour'].min()
end_date = weather_df['time_hour'].max()

# Filter merged data to weather date range
merged_filtered = merged_hourly[
    (merged_hourly['pickup_hour'] >= start_date) & 
    (merged_hourly['pickup_hour'] <= end_date)
]

print(merged_filtered.head())
print(f"Filtered merged data range: {merged_filtered['pickup_hour'].min()} to {merged_filtered['pickup_hour'].max()}")


In [ ]:
# Create rain indicator (True if rain > 0)
merged_filtered['rain_flag'] = merged_filtered['rain (mm)'] > 0

# Count rides in rain vs no rain
rides_by_rain = merged_filtered.groupby('rain_flag')['rides_count'].sum()

print(rides_by_rain)

# Plot rides count for rain vs no rain
import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(x=rides_by_rain.index.astype(str), y=rides_by_rain.values)
plt.title('Total Uber Rides in Rain vs No Rain Hours')
plt.xlabel('Rain')
plt.ylabel('Total Rides')
plt.show()


In [ ]:
merged_filtered.head

In [ ]:
# Count missing values per column
missing_counts = merged_filtered.isnull().sum()

# Percentage of missing values per column
missing_percent = (merged_filtered.isnull().mean() * 100).round(2)

# Combine into one DataFrame for clearer view
missing_summary = pd.DataFrame({'Missing Count': missing_counts, 'Missing %': missing_percent})

print(missing_summary)


In [ ]:
print(merged_filtered['weather_code (wmo code)'].unique())


In [ ]:
weather_impact = merged_filtered.groupby('weather_code (wmo code)').agg({
    'rides_count': 'sum',
    'avg_fare': 'mean'
}).reset_index()

print(weather_impact)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
sns.barplot(x='weather_code (wmo code)', y='rides_count', data=weather_impact)
plt.title('Total Rides by Weather Code')
plt.xlabel('Weather Code (WMO)')
plt.ylabel('Total Rides')

plt.subplot(1,2,2)
sns.barplot(x='weather_code (wmo code)', y='avg_fare', data=weather_impact)
plt.title('Average Fare by Weather Code')
plt.xlabel('Weather Code (WMO)')
plt.ylabel('Average Fare')

plt.tight_layout()
plt.show()


In [ ]:
# Group total rides by weather_code (weather condition)
rides_by_weather = merged_filtered.groupby('weather_code (wmo code)')['rides_count'].sum().reset_index()

# Map weather codes to descriptive text (only for your reference here)
weather_desc = {
    0: 'Clear sky',
    1: 'Mainly clear',
    2: 'Partly cloudy',
    3: 'Overcast',
    45: 'Fog',
    48: 'Depositing rime fog',
    51: 'Light drizzle',
    53: 'Moderate drizzle',
    55: 'Dense drizzle',
    56: 'Light freezing drizzle',
    57: 'Dense freezing drizzle',
    61: 'Slight rain',
    63: 'Moderate rain',
    65: 'Heavy rain',
    66: 'Light freezing rain',
    67: 'Heavy freezing rain',
    71: 'Slight snow fall',
    73: 'Moderate snow fall',
    75: 'Heavy snow fall',
    77: 'Snow grains',
    80: 'Slight rain showers',
    81: 'Moderate rain showers',
    82: 'Violent rain showers',
    85: 'Slight snow showers',
    86: 'Heavy snow showers',
    95: 'Thunderstorm',
    96: 'Thunderstorm with slight hail',
    99: 'Thunderstorm with heavy hail',
}

# Add description column (you can keep or drop this later)
rides_by_weather['weather_desc'] = rides_by_weather['weather_code (wmo code)'].map(weather_desc)

# Filter out all rain-related weather codes (those containing "rain" or drizzle)
rain_terms = ['rain', 'drizzle']

# Filter out rain weather conditions
non_rain_weather = rides_by_weather[~rides_by_weather['weather_desc'].str.contains('|'.join(rain_terms), case=False, na=False)]

# Sort by total rides descending
non_rain_weather = non_rain_weather.sort_values(by='rides_count', ascending=False)

print(non_rain_weather[['weather_desc', 'rides_count']])


In [ ]:
import matplotlib.pyplot as plt

# Assuming you already have 'non_rain_weather' DataFrame from previous step

plt.figure(figsize=(12,6))
plt.bar(non_rain_weather['weather_desc'], non_rain_weather['rides_count'], color='skyblue')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Weather Condition')
plt.ylabel('Total Rides')
plt.title('Total Uber Rides by Weather Condition (excluding rain/drizzle)')
plt.tight_layout()
plt.show()


In [ ]:
print(uber_df.columns)


In [ ]:
print(merged_filtered.columns)

In [ ]:
import pandas as pd

# Step 1: Convert datetime columns and remove timezone (tz_localize(None))
uber_df['pickup_hour'] = pd.to_datetime(uber_df['pickup_hour']).dt.tz_localize(None)
weather_df['time_hour'] = pd.to_datetime(weather_df['time']).dt.floor('h').dt.tz_localize(None)

# Step 2: Filter Uber data within weather data's time range
start = weather_df['time_hour'].min()
end = weather_df['time_hour'].max()

uber_filtered = uber_df[
    (uber_df['pickup_hour'] >= start) &
    (uber_df['pickup_hour'] <= end)
].copy()

# Step 3: Optional - round location to reduce unique location combinations
uber_filtered['pickup_lat_rounded'] = uber_filtered['pickup_latitude'].round(2)
uber_filtered['pickup_lon_rounded'] = uber_filtered['pickup_longitude'].round(2)

# Step 4: Group by time and location
grouped_uber = uber_filtered.groupby(
    ['pickup_hour', 'pickup_lat_rounded', 'pickup_lon_rounded']
).agg(
    rides_count=('fare_amount', 'count'),
    avg_fare=('fare_amount', 'mean')
).reset_index()

# Step 5: Merge ride + weather data
merged_full = pd.merge(
    grouped_uber,
    weather_df,
    left_on='pickup_hour',
    right_on='time_hour',
    how='inner'
)

# Step 6: Drop unneeded columns
merged_full.drop(columns=['time', 'time_hour'], inplace=True)

# Final check
print(merged_full.head())
print(merged_full.columns)


In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(
    x='pickup_lon_rounded',
    y='pickup_lat_rounded',
    size='rides_count',
    hue='rides_count',
    data=merged_full,
    palette='viridis',
    sizes=(20, 200),
    alpha=0.7,
    legend='brief'
)
plt.title('Rides Count by Location')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(title='Rides Count')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,8))

# Create the scatterplot and capture the PathCollection object
scatter = sns.scatterplot(
    x='pickup_lon_rounded',
    y='pickup_lat_rounded',
    hue='temperature_2m (°C)',
    data=merged_full,
    palette='coolwarm',
    alpha=0.7,
    legend=None  # No legend, we will use colorbar
)

plt.title('Temperature by Location')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Get the scatter plot's collection of points
# This object knows the colormap and normalization
plt.colorbar(scatter.collections[0], label='Temperature (°C)')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter for valid NYC coordinates
merged_full_cleaned = merged_full[
    (merged_full['pickup_lat_rounded'].between(40.5, 41.0)) &
    (merged_full['pickup_lon_rounded'].between(-74.3, -73.6))
]

# Quick check: what ride counts do we have?
print(merged_full_cleaned['rides_count'].describe())

# Plot
plt.figure(figsize=(14, 10))
scatter = sns.scatterplot(
    x='pickup_lon_rounded',
    y='pickup_lat_rounded',
    size='rides_count',
    hue='relative_humidity_2m (%)',
    data=merged_full_cleaned,
    palette='Blues',
    sizes=(10, 600),  # Increased max bubble size for clarity
    alpha=0.7,
    edgecolor='gray',
    linewidth=0.5,
    legend='brief'
)

# Add text labels with ride count on the largest points
for i, row in merged_full_cleaned.iterrows():
    if row['rides_count'] >= 10:  # Only label if large enough to avoid clutter
        plt.text(
            row['pickup_lon_rounded'],
            row['pickup_lat_rounded'],
            str(int(row['rides_count'])),
            color='black',
            fontsize=8,
            ha='center',
            va='center'
        )

# Titles and labels
plt.title('NYC Rides Count and Humidity by Location', fontsize=16)
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Fix legend overlap
handles, labels = scatter.get_legend_handles_labels()
scatter.legend(
    handles=handles,
    labels=labels,
    title='Humidity (%) & Rides Count',
    bbox_to_anchor=(1.05, 1),
    loc='upper left'
)

plt.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()


In [ ]:
uber_df.columns


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Haversine function ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    distance_km = R * c
    return distance_km

# --- Compute trip distances ---
uber_df['trip_distance_km'] = haversine(
    uber_df['pickup_latitude'], 
    uber_df['pickup_longitude'], 
    uber_df['dropoff_latitude'], 
    uber_df['dropoff_longitude']
)

# --- Data cleaning ---
# Remove unrealistic distances
uber_df = uber_df[uber_df['trip_distance_km'] < 100]

# Remove negative or zero fares
uber_df = uber_df[uber_df['fare_amount'] > 0]

# Remove entries with 0 distance and non-zero fare
uber_df = uber_df[~((uber_df['trip_distance_km'] <= 0) & (uber_df['fare_amount'] > 0))]

# Optional: remove 0 distance + 0 fare
uber_df = uber_df[~((uber_df['trip_distance_km'] == 0) & (uber_df['fare_amount'] == 0))]

# --- Plot ---
plt.figure(figsize=(10, 6))
sns.scatterplot(x='trip_distance_km', y='fare_amount', data=uber_df, alpha=0.3)

plt.title('Fare Amount vs Trip Distance (km) [Cleaned]')
plt.xlabel('Trip Distance (km)')
plt.ylabel('Fare Amount (USD)')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Select numeric columns and exclude index-like columns
numeric_cols = uber_df.select_dtypes(include=['number']).drop(columns=['Unnamed: 0'], errors='ignore')

# Compute correlation matrix
corr_matrix = numeric_cols.corr()

# Plot heatmap with masked upper triangle
plt.figure(figsize=(len(corr_matrix.columns), len(corr_matrix.columns)))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(
    corr_matrix, 
    mask=mask, 
    annot=True, 
    cmap='coolwarm', 
    fmt='.2f', 
    linewidths=0.5,
    cbar_kws={'shrink': 0.8}
)
plt.title('Correlation Matrix for Numeric Columns in uber_df', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()  # Prevent label cutoff
plt.show()

In [ ]:
import pandas as pd

# 1. Work on a copy of the dataset
df = merged_full.copy()

# 2. Convert 'pickup_hour' to datetime
df['pickup_hour'] = pd.to_datetime(df['pickup_hour'])

# 3. Drop rows with any missing (NaN) values
df.dropna(inplace=True)

# 4. Remove rows with non-positive or extreme values if needed
df = df[df['rides_count'] > 0]
df = df[df['avg_fare'] > 0]

# 5. Filter latitude/longitude for NYC region (if needed)
df = df[(df['pickup_lat_rounded'] >= 40.5) & (df['pickup_lat_rounded'] <= 41.0)]
df = df[(df['pickup_lon_rounded'] >= -74.3) & (df['pickup_lon_rounded'] <= -73.6)]

# 6. Optional: Check for duplicates
df.drop_duplicates(inplace=True)

# 7. Reset index
df.reset_index(drop=True, inplace=True)

# Summary
print("Cleaned data shape:", df.shape)
df.head()


In [ ]:
# Extract hour and day of week from pickup_hour
df['hour'] = df['pickup_hour'].dt.hour
df['day_of_week'] = df['pickup_hour'].dt.dayofweek  # Monday=0, Sunday=6

# Average rides_count by hour
hourly_rides = df.groupby('hour')['rides_count'].mean()

plt.figure(figsize=(10,5))
hourly_rides.plot(kind='line', marker='o')
plt.title('Average Number of Rides by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Rides Count')
plt.grid(True)
plt.show()

# Average fare by day of week
day_fare = df.groupby('day_of_week')['avg_fare'].mean()

plt.figure(figsize=(10,5))
day_fare.plot(kind='bar')
plt.title('Average Fare by Day of Week')
plt.xlabel('Day of Week (0=Mon)')
plt.ylabel('Average Fare ($)')
plt.show()


In [ ]:
# Summary statistics for numeric columns
print(df.describe())

# Check distribution of rides_count and avg_fare
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.histplot(df['rides_count'], bins=50, kde=True)
plt.title('Distribution of Rides Count')

plt.subplot(1,2,2)
sns.histplot(df['avg_fare'], bins=50, kde=True)
plt.title('Distribution of Average Fare')
plt.show()


In [ ]:
# Group by date
df['date'] = df['pickup_hour'].dt.date
daily_rides = df.groupby('date')['rides_count'].sum()

plt.figure(figsize=(12,6))
daily_rides.plot()
plt.title('Total Daily Rides Over Time')
plt.xlabel('Date')
plt.ylabel('Total Rides Count')
plt.show()


# 🚗 Uber Traffic Analysis Summary

## ✅ Findings
- **Rides drop** in bad weather:  
  - Good weather: 173,977 rides  
  - Bad weather: 26,023 rides  
- **Fare amount stays similar**:  
  - Good weather: $11.37  
  - Bad weather: $11.27  
- **Holiday status** has little to no effect.

## 📚 Learnings
- Bad weather reduces Uber usage.
- Fare is not strongly affected by weather.
- Holidays don’t impact fare or ride volume much.

## 🛠️ What Can Be Built
- Ride demand prediction model using weather.
- Heatmaps of ride frequency vs weather.
- Dashboards or alert systems for low-demand periods.


In [19]:
print(uber_df.columns)


Index(['Unnamed: 0', 'key', 'fare_amount', 'pickup_datetime',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count'],
      dtype='object')
